In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
 import spacy
 from spacy.tokens import DocBin
 from tqdm import tqdm

 nlp = spacy.blank("en") # load a new spacy model
 db = DocBin() # create a DocBin object

In [5]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/downcy_symp/annotations.json')
TRAIN_DATA = json.load(f)

In [ ]:
TRAIN_DATA

In [6]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot["entities"]:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

  db.to_disk("/content/drive/MyDrive/Colab Notebooks/downcy_symp/training_data.spacy")

100%|██████████| 201/201 [00:03<00:00, 66.00it/s]


In [7]:
! python -m spacy init config "/content/drive/MyDrive/Colab Notebooks/downcy_symp/config.cfg" --lang en --pipeline ner --optimize efficiency

2023-09-28 09:24:49.446610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Colab Notebooks/downcy_symp/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
!python -m spacy train "/content/drive/MyDrive/Colab Notebooks/downcy_symp/config.cfg" --output ./ --paths.train="/content/drive/MyDrive/Colab Notebooks/downcy_symp/training_data.spacy" --paths.dev="/content/drive/MyDrive/Colab Notebooks/downcy_symp/training_data.spacy"

2023-09-28 09:27:45.817609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     35.40    0.00    0.00    0.00    0.00
  2     200       2545.59   3709.72   45.52   47.10   44.03    0.46
  5     400        235.73    919.10   86.80   86.61   86.98    0.87
  8     600        408.23    575.84   92.37   92.78   91.97    0.92
 12     800        224.34    325.78   95.98   96.09   95.88    0.96
 18    1000        225.68    301.10   96.73   97.16   96.31    0.97

In [12]:
nlp_ner = spacy.load("/content/model-last")

In [13]:
doc = nlp_ner(''' I was watching a health documentary, and they mentioned kids with an unusually small head. It struck me because my child has the same. Can you advise?  ''')

In [14]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [25]:
import zipfile
import os

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Specify the folder path and output path
folder_path = '/content/drive/MyDrive/Colab Notebooks/downcy_symp/'  # Replace with your folder path
output_path = '/content/drive/MyDrive/Colab Notebooks/downcy_symp.zip'  # Replace with your desired output path

# Zip the folder
zip_folder(folder_path, output_path)

In [30]:
import shutil
import zipfile
import os

# Source folder path to be zipped
source_folder_path = '/content/model-last'

# Destination zip file path
destination_zip_path = '/content/drive/MyDrive/downcy_symp/model-last.zip'

# Zip the source folder
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Zip the folder
zip_folder(source_folder_path, destination_zip_path)

# Verify if the zip file has been created successfully
if os.path.exists(destination_zip_path):
    print(f'The folder has been successfully zipped and saved to: {destination_zip_path}')
else:
    print('Failed to zip the folder.')


The folder has been successfully zipped and saved to: /content/drive/MyDrive/downcy_symp/model-last.zip
